In [74]:
import os
import numpy as np
import pandas as pd
from lsst.sims.maf.db import ResultsDb
from IPython.display import display, HTML

In [61]:
newruns = ['minion_1016',
           'astro_lsst_01_1016',
           'astro_lsst_01_1015',
           'astro_lsst_01_1017']
oldruns = ['enigma_1189', 'enigma_1281', 'enigma_1282', 'minion_1016']

In [62]:
pha_new = {}
neo_new = {}
for r in newruns:
    pha_new[r] = ResultsDb(os.path.join('.', r, 'pha_newsky'))
    neo_new[r] = ResultsDb(os.path.join('.', r, 'neo_newsky'))

In [63]:
pha_old = {}
neo_old = {}
for r in oldruns:
    pha_old[r] = ResultsDb(os.path.join('.', r, 'pha'))
    neo_old[r] = ResultsDb(os.path.join('.', r, 'neo'))

In [64]:
def make_dataframe(runs, runlist, metricName='CumulativeCompleteness'):
    summary = []
    for r in runlist:
        mId = runs[r].getMetricId(metricName)
        s = runs[r].getSummaryStats(mId, summaryName=None)
        summary.append(pd.DataFrame(zip(s['metricMetadata'], s['summaryValue']*100.0), columns=['metadata', r]))
    df = summary[0]
    for i in range(1, len(summary)):
        df = pd.merge(df, summary[i], how='outer', on='metadata')
    return df

In [65]:
pha_new_df = make_dataframe(pha_new, newruns)
pha_old_df = make_dataframe(pha_old, oldruns)
neo_new_df = make_dataframe(neo_new, newruns)
neo_old_df = make_dataframe(neo_old, oldruns)

In [66]:
def show_match(metadata, df):
    display(df[df.metadata.str.contains(metadata)])

In [67]:
def show_diff(metadata1, metadata2, df):
    runnames = list(df.columns.values)
    runnames.remove('metadata')
    d = (df[df.metadata.str.contains(metadata2)][runnames].as_matrix() - 
         df[df.metadata.str.contains(metadata1)][runnames].as_matrix())
    diff = pd.DataFrame(d, columns=runnames)
    diff['average'] = np.nanmean(d)
    print "Going from %s to %s:" % (metadata1, metadata2)
    display(diff)

In [68]:
df = pha_new_df
match1 = '3 pairs in 15 nights yr 10'
show_match(match1, df)
match15 = '3 pairs in 30 nights yr 10'
show_match(match15, df)
show_diff(match1, match15, df)
match2 = '3 pairs in 15 nights yr 12'
show_match(match2, df)
show_diff(match1, match2, df)

match3 = '3 pairs in 30 nights yr 12'
show_match(match3, df)
show_diff(match2, match3, df)

match4 = '3 pairs in 30 nights trailing loss yr 12'
show_match(match4, df)
show_diff(match3, match4, df)

match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 12'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017
7,PHA 3 pairs in 15 nights yr 10,73.7,74.8,73.6,76.2


,metadata,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017
43,PHA 3 pairs in 30 nights yr 10,76.0,76.8,77.4,79.0


Going from 3 pairs in 15 nights yr 10 to 3 pairs in 30 nights yr 10:


,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017,average
0,2.2,2.0,3.8,2.8,2.7


,metadata,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017
141,PHA 3 pairs in 15 nights yr 12,--,79.1,77.5,79.9


...

Going from 3 pairs in 30 nights yr 12 to 3 pairs in 30 nights trailing loss yr 12:


,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017,average
0,--,0.6,1.9,1.5,1.3


,metadata,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017
156,"PHA 3 pairs in 30 nights trailing loss, SNR=4 yr 12",--,83.8,85.6,86.0


Going from 3 pairs in 30 nights trailing loss yr 12 to 3 pairs in 30 nights trailing loss, SNR=4 yr 12:


,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017,average
0,--,2.3,2.8,2.1,2.4


In [10]:
df = neo_new_df
match1 = '3 pairs in 15 nights yr 10'
show_match(match1, df)
match15 = '3 pairs in 30 nights yr 10'
show_match(match15, df)
show_diff(match1, match15, df)
match2 = '3 pairs in 15 nights yr 12'
show_match(match2, df)
show_diff(match1, match2, df)
match3 = '3 pairs in 30 nights yr 12'
show_match(match3, df)
show_diff(match2, match3, df)
match4 = '3 pairs in 30 nights trailing loss yr 12'
show_match(match4, df)
show_diff(match3, match4, df)
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 12'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
17,NEO 3 pairs in 15 nights yr 10,60.836294,57.718595,62.42445,56.811208


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
106,NEO 3 pairs in 30 nights yr 10,63.38195,61.847965,65.166977,59.595336


Going from 3 pairs in 15 nights yr 10 to 3 pairs in 30 nights yr 10:


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,2.545656,4.129371,2.742527,2.784128,3.050421


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
18,NEO 3 pairs in 15 nights yr 12,60.836294,61.361651,65.922897,60.41197


...

Going from 3 pairs in 30 nights yr 12 to 3 pairs in 30 nights trailing loss yr 12:


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,0.790176,1.928089,1.694449,0.744518,1.289308


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
93,"NEO 3 pairs in 30 nights trailing loss, SNR=4 ...",67.537108,71.502199,73.616514,67.030553


Going from 3 pairs in 30 nights trailing loss yr 12 to 3 pairs in 30 nights trailing loss, SNR=4 yr 12:


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,3.364982,4.055641,3.259152,3.146947,3.45668


In [11]:
df = pha_old_df
match2 = '3 pairs in 15 nights yr 10'
show_match(match2, df)
#show_diff(match1, match2, df)
match3 = '3 pairs in 30 nights yr 10'
show_match(match3, df)
show_diff(match2, match3, df)
match4 = '3 pairs in 30 nights trailing loss yr 10'
show_match(match4, df)
show_diff(match3, match4, df)
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 10'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,PHA 3 pairs in 15 nights yr 10,72.286809,70.064613,74.964997,67.029049


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
43,PHA 3 pairs in 30 nights yr 10,74.810999,73.124718,77.59089,71.435025


Going from 3 pairs in 15 nights yr 10 to 3 pairs in 30 nights yr 10:


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,2.52419,3.060105,2.625893,4.405976,3.154041


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
31,PHA 3 pairs in 30 nights trailing loss yr 10,75.24956,73.645192,78.012546,72.167027


Going from 3 pairs in 30 nights yr 10 to 3 pairs in 30 nights trailing loss yr 10:


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,0.438562,0.520475,0.421656,0.732002,0.528173


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
37,"PHA 3 pairs in 30 nights trailing loss, SNR=4 ...",77.961248,76.531139,80.638334,74.686238


Going from 3 pairs in 30 nights trailing loss yr 10 to 3 pairs in 30 nights trailing loss, SNR=4 yr 10:


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,2.711687,2.885947,2.625788,2.519211,2.685658


In [12]:
# compare NEO completeness to PHA completeness
def show_two(metadata, df1, df2):
    runnames1 = set(df1.columns.values)
    runnames2 = set(df2.columns.values)
    runnames = runnames1.intersection(runnames2)
    runnames = list(runnames)
    runnames.remove('metadata')
    d = (df2[df2.metadata.str.contains(metadata)][runnames].as_matrix() - 
         df1[df1.metadata.str.contains(metadata)][runnames].as_matrix())
    diff = pd.DataFrame(d, columns=runnames)
    diff['average'] = np.nanmean(d)
    display(diff)

match = '3 pairs in 15 nights yr 10'
df1 = pha_new_df
df2 = neo_new_df
show_match(match, df1)
show_match(match, df2)
show_two(match, df1, df2)

,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
7,PHA 3 pairs in 15 nights yr 10,73.729326,73.612485,76.205749,74.829865


,metadata,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016
17,NEO 3 pairs in 15 nights yr 10,60.836294,57.718595,62.42445,56.811208


,minion_1016,astro_lsst_01_1015,astro_lsst_01_1017,astro_lsst_01_1016,average
0,-12.893032,-15.893891,-13.781299,-18.018657,-15.14672


In [13]:
df = neo_old_df
match2 = '3 pairs in 15 nights yr 10'
show_match(match2, df)
#show_diff(match1, match2, df)
match3 = '3 pairs in 30 nights yr 10'
show_match(match3, df)
show_diff(match2, match3, df)
match4 = '3 pairs in 30 nights trailing loss yr 10'
show_match(match4, df)
show_diff(match3, match4, df)
match5 = '3 pairs in 30 nights trailing loss, SNR=4 yr 10'
show_match(match5, df)
show_diff(match4, match5, df)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,NEO 3 pairs in 15 nights yr 10,59.461635,57.851317,64.833056,54.657858


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
43,NEO 3 pairs in 30 nights yr 10,62.424463,61.716776,68.361687,59.231282


Going from 3 pairs in 15 nights yr 10 to 3 pairs in 30 nights yr 10:


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,2.962827,3.865459,3.528631,4.573424,3.732585


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
31,NEO 3 pairs in 30 nights trailing loss yr 10,63.049837,62.327055,68.633961,59.775857


Going from 3 pairs in 30 nights yr 10 to 3 pairs in 30 nights trailing loss yr 10:


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,0.625375,0.610278,0.272274,0.544575,0.513125


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
37,"NEO 3 pairs in 30 nights trailing loss, SNR=4 ...",66.892363,65.42489,71.976552,62.927365


Going from 3 pairs in 30 nights trailing loss yr 10 to 3 pairs in 30 nights trailing loss, SNR=4 yr 10:


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,3.842526,3.097835,3.342591,3.151508,3.358615


In [14]:
df = neo_old_df
match1 = '3 pairs in 15 nights yr 10'
show_match(match1, df)
match2 = '3 pairs in 30 nights yr 10'
show_match(match2, df)
match3 = '3 triplets in 30 nights yr 10'
show_match(match3, df)
match4 = '3 quads in 30 nights yr 10'
show_match(match4, df)
match5 = '3 pairs in 30 nights trailing loss yr 10'
show_match(match5, df)
match6 = '3 pairs in 30 nights trailing loss, SNR=4 yr 10'
show_match(match6, df)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,NEO 3 pairs in 15 nights yr 10,59.461635,57.851317,64.833056,54.657858


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
43,NEO 3 pairs in 30 nights yr 10,62.424463,61.716776,68.361687,59.231282


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
79,NEO 3 triplets in 30 nights yr 10,39.739229,53.290773,44.497952,52.426873


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
61,NEO 3 quads in 30 nights yr 10,27.15548,25.700641,31.261804,43.548584


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
31,NEO 3 pairs in 30 nights trailing loss yr 10,63.049837,62.327055,68.633961,59.775857


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
37,"NEO 3 pairs in 30 nights trailing loss, SNR=4 ...",66.892363,65.42489,71.976552,62.927365


In [15]:
# compare NEO completeness to PHA completeness
def show_two(metadata, df1, df2):
    runnames1 = set(df1.columns.values)
    runnames2 = set(df2.columns.values)
    runnames = runnames1.intersection(runnames2)
    runnames = list(runnames)
    runnames.remove('metadata')
    d = (df2[df2.metadata.str.contains(metadata)][runnames].as_matrix() - 
         df1[df1.metadata.str.contains(metadata)][runnames].as_matrix())
    diff = pd.DataFrame(d, columns=runnames)
    diff['average'] = np.nanmean(d)
    display(diff)

match = '3 pairs in 15 nights yr 10'
df1 = pha_old_df
df2 = neo_old_df
show_match(match, df1)
show_match(match, df2)
show_two(match, df1, df2)

,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,PHA 3 pairs in 15 nights yr 10,72.286809,70.064613,74.964997,67.029049


,metadata,minion_1016,enigma_1281,enigma_1189,enigma_1282
7,NEO 3 pairs in 15 nights yr 10,59.461635,57.851317,64.833056,54.657858


,minion_1016,enigma_1281,enigma_1189,enigma_1282,average
0,-12.825174,-12.213296,-10.131942,-12.371191,-11.8854


In [76]:
df = pha_new_df
criteria = ['PHA 3 pairs in 15 nights yr 10', 'PHA 3 pairs in 30 nights yr 10', 'PHA 3 pairs in 30 nights trailing loss yr 10',
            'PHA 3 pairs in 15 nights yr 12', 'PHA 3 pairs in 30 nights yr 12', 
            'PHA 3 pairs in 30 nights trailing loss yr 12', 'PHA 3 pairs in 30 nights trailing loss, SNR=4 yr 12']
c = criteria[0]
d = df.query('metadata == @c')
for c in criteria[1:]:
    d2 = df.query('metadata == @c')
    d = pd.concat([d, d2])
vals = d.columns.values
vals[0] = 'Discovery Criteria'
d.columns = vals

In [77]:
pd.set_option('max_colwidth', 65)
def float_format(x):
    if np.isnan(x):
        return '--'
    else:
        return '%.1f' %x
pd.set_option('float_format', float_format)
display(HTML(d.to_html(index=False)))

Discovery Criteria,minion_1016,astro_lsst_01_1016,astro_lsst_01_1015,astro_lsst_01_1017
PHA 3 pairs in 15 nights yr 10,73.7,74.8,73.6,76.2
PHA 3 pairs in 30 nights yr 10,76.0,76.8,77.4,79.0
PHA 3 pairs in 30 nights trailing loss yr 10,76.5,77.4,79.5,80.6
PHA 3 pairs in 15 nights yr 12,--,79.1,77.5,79.9
PHA 3 pairs in 30 nights yr 12,--,80.8,80.9,82.5
PHA 3 pairs in 30 nights trailing loss yr 12,--,81.5,82.9,83.9
"PHA 3 pairs in 30 nights trailing loss, SNR=4 yr 12",--,83.8,85.6,86.0
